In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import re

import spacy
from pymystem3 import Mystem
from nltk.corpus import stopwords
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer

import string
import time

In [2]:
arr = os.listdir('./input/')
arr_df = pd.DataFrame(arr, columns = ['file_name'])
arr_df

,file_name
0,14773455.jl
1,file_7321705_455951711_124.jl
2,file_11834040_455951711_112.jl
3,w2.jl
4,file_10094987_455951711_145.jl
5,w3.jl
6,17597852.jl
7,5044562.jl
8,file_7961412_455951711_109.jl
9,file_6170053_455951711_115.jl


In [63]:
file_index = 23
path = f'./input/{arr_df["file_name"][file_index]}'

In [64]:
columns = ['comment', 'date_time', 'color','size', 'thumb_up', 'thumb_down', 'prod_eval', 'prod', 'brand']

df = pd.read_json(path).transpose().reset_index().drop('index', axis=1)
df = df.set_axis(columns, axis = 'columns')

df.head(5)

,comment,date_time,color,size,thumb_up,thumb_down,prod_eval,prod,brand
0,"Лезвия очень тупые! На упаковке одно, а внутри...",2021-05-02T20:36:11Z,,0,1,0,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
1,Лезвия тупые. Не рекомендую.,2021-05-02T16:18:13Z,,0,2,0,3,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
2,"Отвратительные станки, лезвие тупые, не бреют....",2021-04-27T13:53:37Z,,0,1,1,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
3,"Лезвия оригинальные,произведено в Колумбии.Все...",2021-04-27T11:06:13Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
4,"Сменные кассеты для бритья мужу понравились, б...",2021-04-27T09:41:42Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE


# Функции: Clean_text, Stopwords_del

In [65]:
def delete_non_letters(words):
    new_words = []
    words = words.split()
    
    for word in words:
        new_word = "".join(c if c.isalpha() else " " for c in word )
        
        if new_word != '':
            new_words.append(new_word)
    text = ' '.join(c for c in new_words)
        
    return text

In [66]:
### Text Normalizing function. Part of the following function was taken from this link. 
def clean_text(text):
    
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U0001F1F2-\U0001F1F4"  # Macau flag
        u"\U0001F1E6-\U0001F1FF"  # flags
        u"\U0001F600-\U0001F64F"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U0001F1F2"
        u"\U0001F1F4"
        u"\U0001F620"
        u"\u200d"
        u"\u2640-\u2642"
        "]+", flags=re.UNICODE)

    #удаляет пунктуацию
    #translation_table = str.maketrans("", "", string.punctuation)
    #text = text.translate(translation_table)   
    
    text = text.lower()
    ## Clean the text
    text = re.sub(r"[,_»«\*!.\/'+-=)(]", " ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\%", " ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"\|", " ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r'"', " ", text)
    text = re.sub(r'«', " ", text)
    text = re.sub(r'\*', " ", text)
    text = re.sub(r'\?', " ", text)
    text = re.sub(r'»', " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r"\s{2,}", " ", text)
    text = emoji_pattern.sub(r'', text)
    #text = " ".join(text.split())
    
    text = delete_non_letters(text)
    
    return text


In [67]:
def delete_stopwords(text): 
    text = text.split()
    text = [w for w in text if not w in russian_stopwords and len(w) >= 3]
    text = " ".join(text)
    
    return text

In [68]:
df.head()

,comment,date_time,color,size,thumb_up,thumb_down,prod_eval,prod,brand
0,"Лезвия очень тупые! На упаковке одно, а внутри...",2021-05-02T20:36:11Z,,0,1,0,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
1,Лезвия тупые. Не рекомендую.,2021-05-02T16:18:13Z,,0,2,0,3,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
2,"Отвратительные станки, лезвие тупые, не бреют....",2021-04-27T13:53:37Z,,0,1,1,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
3,"Лезвия оригинальные,произведено в Колумбии.Все...",2021-04-27T11:06:13Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE
4,"Сменные кассеты для бритья мужу понравились, б...",2021-04-27T09:41:42Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE


# Stopwords extension

In [69]:
file_name = 'StopWords_extension.csv'
path = f'./{file_name}'
SW = pd.read_csv(path, index_col = 'Index')
SW_list = SW['Word'].tolist()

In [70]:
prod_text = clean_text(df['prod'][0])
brand_text = clean_text(df['brand'][0])

In [71]:
stopwords_add_by_hand = ['цена','пломба','классный','доставка','очень','довольный', 'быстрый','быстро', 'хороший','отлично','все','прийти', 'класс','отличный', 'свой', 'отзыв', 'приходить', 'супер','это', 'спасибо', 'работа']
stopwords_add_by_category = [i for i in brand_text.split()]+[i for i in prod_text.split()]
stopwords_add_by_category

['gillette', 'сменные', 'кассеты', 'для', 'бритья', 'fusion', 'шт']

In [72]:
russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(stopwords_add_by_hand+stopwords_add_by_category+SW_list)

# Clean comments before lemmatization

In [73]:
%time
df['cleaned_comment'] = df['comment'].map(lambda x: clean_text(x))

CPU times: user 4 µs, sys: 1 µs, total: 5 µs
Wall time: 24.1 µs


In [74]:
#df['lemma_comment'] = df['cleaned_comment'].map(lambda x: delete_stopwords(x))

In [75]:
df.head(5)

,comment,date_time,color,size,thumb_up,thumb_down,prod_eval,prod,brand,cleaned_comment
0,"Лезвия очень тупые! На упаковке одно, а внутри...",2021-05-02T20:36:11Z,,0,1,0,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,лезвия очень тупые на упаковке одно а внутри с...
1,Лезвия тупые. Не рекомендую.,2021-05-02T16:18:13Z,,0,2,0,3,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,лезвия тупые не рекомендую
2,"Отвратительные станки, лезвие тупые, не бреют....",2021-04-27T13:53:37Z,,0,1,1,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,отвратительные станки лезвие тупые не бреют не...
3,"Лезвия оригинальные,произведено в Колумбии.Все...",2021-04-27T11:06:13Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,лезвия оригинальные произведено в колумбии все...
4,"Сменные кассеты для бритья мужу понравились, б...",2021-04-27T09:41:42Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,сменные кассеты для бритья мужу понравились бу...


# Mystem lemmatization and drop empty comments

In [76]:
df.shape

(1186, 10)

In [77]:
df = df.drop(df[df['cleaned_comment']==''].index).reset_index(drop = True)

In [78]:
def lemmatize(text): 
    text =  [i for i in text]
    m = Mystem()
    merged_text = "|".join(text)

    doc = []
    res = []
    count = 0
    lemma = m.lemmatize(merged_text)
    for t in lemma:
        
        if '|' not in t and count+1<len(lemma):
            doc.append(t)
            count+=1
          
        else:
            doc = ''.join(i for i in doc)
            res.append(doc)
            count+=1
            doc = []
    return res

In [79]:
%time
res = lemmatize(df['cleaned_comment'])

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 7.15 µs


In [80]:
df['lemma_comment'] = res

# Delete StopWords

In [81]:
%time
df['lemma_comment'] = df['lemma_comment'].map(lambda x: delete_stopwords(x))
df = df.drop(df[df['lemma_comment']==''].index)

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 4.77 µs


In [82]:
df.head(5)

,comment,date_time,color,size,thumb_up,thumb_down,prod_eval,prod,brand,cleaned_comment,lemma_comment
0,"Лезвия очень тупые! На упаковке одно, а внутри...",2021-05-02T20:36:11Z,,0,1,0,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,лезвия очень тупые на упаковке одно а внутри с...,лезвие тупой упаковка внутри насадка рекомендо...
1,Лезвия тупые. Не рекомендую.,2021-05-02T16:18:13Z,,0,2,0,3,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,лезвия тупые не рекомендую,лезвие тупой рекомендовать
2,"Отвратительные станки, лезвие тупые, не бреют....",2021-04-27T13:53:37Z,,0,1,1,1,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,отвратительные станки лезвие тупые не бреют не...,отвратительный станок лезвие тупой брить реком...
3,"Лезвия оригинальные,произведено в Колумбии.Все...",2021-04-27T11:06:13Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,лезвия оригинальные произведено в колумбии все...,лезвие оригинальный производить колумбия призн...
4,"Сменные кассеты для бритья мужу понравились, б...",2021-04-27T09:41:42Z,,0,1,0,5,"Сменные кассеты для бритья FUSION, 2 шт.",GILLETTE,сменные кассеты для бритья мужу понравились бу...,сменный кассета бритье муж понравиться заказыв...


In [83]:
df = df.reset_index(drop = True)

In [84]:
preprocessed_comments = df['lemma_comment']

# tf-idf

In [85]:
vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2))

In [86]:
vectorized_comments = vectorizer.fit_transform(preprocessed_comments)

In [87]:
#  creating a dictionary mapping the tokens to their tfidf values
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
tfidf = pd.DataFrame(columns=['tfidf']).from_dict(
                    dict(tfidf), orient='index')
tfidf.columns = ['tfidf']

In [88]:
#Эти слова показываем пользователю, он вводит то, по чему хочет почитать подробнее, или свое слово
tfidf.sort_values(by=['tfidf'], ascending=True).head(5)

,tfidf
лезвие,2.402526
муж,2.746297
кассета,2.761801
магазин,2.837513
брить,3.070537


In [89]:
tfidf['tfidf']

venus                6.633002
venus возврат        6.920684
wildberries          5.822072
абсолютно            6.920684
абсолютно кассета    6.920684
                       ...   
щетина жесткий       6.409859
экономия             6.920684
явно                 6.227537
явно подделка        6.920684
явный                6.920684
Name: tfidf, Length: 892, dtype: float64

# Looking for most similar comments for each word

In [90]:
nlp = spacy.load('ru_core_news_lg')

In [91]:
def top_10_similar(word_for_checking, dataframe):
    critical_similarity_value = 0.44
    
    word_for_checking = nlp(word_for_checking)
    similarities = []
    for i in range(len(dataframe['lemma_comment'])):
        similarities.append(nlp(dataframe['lemma_comment'][i]).similarity(word_for_checking))
    
    df_temp = dataframe.copy()
    
    df_temp[f'similarity_to_{word_for_checking}'] = similarities
    #сортировка по убыванию similarities, фильтрация в соответствии с critical_similarity_value
    df_temp = df_temp.sort_values(by = f'similarity_to_{word_for_checking}', ascending = False).head(10)
    res = df_temp[df_temp[f'similarity_to_{word_for_checking}'] > critical_similarity_value][['comment', f'similarity_to_{word_for_checking}']]
    res = list(res['comment'])
    
    if len(res)>0:
        return res
    else: 
        return "По вашему запросу совпадений не найдено"

In [92]:
#%time
#start_time = time.time()
top_10_similar('цена', df)
#print("--- %s seconds ---" % (time.time() - start_time))

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  import sys


['Хорошего качества. Покупкой доволен',
 'Отличный товар за 452 руб. 👍',
 'Взял и не пожалел, оригинал в магазине со скидкой стоили дороже.',
 'Упаковка целая. Со скидкой вышло 573 рубля. Покупкой довольны.',
 'Ну про Джилетт можно и ни чего и не писать качество продукции говорит само за себя, за 407 рублей два лезвия это супер 👍 у нас в городе они стоят 480-490 рублей,упаковку не сохранил но она пришла целая без повреждений! Спасибо большое производителям и Вайлдберриз!!!',
 'Переживала, что качество будет не очень, но муж остался доволен. А если учесть, что цена со скидкой почти даром, относительно стоимости в магазинах Минска, то это просто бомба)))',
 'Кассеты хорошие совершенно не отличаются от купленных ранее в магазине за стоимость намного выше, посмотрим насколько хватет.Кассеты подошли для станка с движующей головкой. \nЕдинственный минус дата выпуска 2017 год!',
 'Товар получил все норма',
 'Вот такая фигня после первого использования. Лучше потратить на 100 руб больше, но ку